<a href="https://colab.research.google.com/github/Sewanmpm/SlumImprovment/blob/main/Segmentation/imageSubdivisioncorrect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from shapely.geometry import Polygon
!pip install leafmap
import leafmap

In [5]:
%pip install geopy
%pip install geopandas






In [55]:
def create_polygon_from_bbox(min_point, max_point):
    """
    Create a polygon (rectangle) from a bounding box defined by minimum and maximum points.

    Parameters:
    - min_point (tuple): The minimum point (longitude, latitude) of the bounding box.
    - max_point (tuple): The maximum point (longitude, latitude) of the bounding box.

    Returns:
    - list of tuples: Four vertices of the rectangle defined by the bounding box, in clockwise order.
    """
    min_lon, min_lat = min_point
    max_lon, max_lat = max_point

    # Define the four corners of the rectangle
    bottom_left = (min_lon, min_lat)
    bottom_right = (max_lon, min_lat)
    top_right = (max_lon, max_lat)
    top_left = (min_lon, max_lat)

    # List the vertices in clockwise order
    vertices = [bottom_left, bottom_right, top_right, top_left]

    return vertices

In [56]:
import geopy.distance



In [57]:
import math

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in meters between two points
    on the earth (specified in decimal degrees).
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371000 # Radius of Earth in meters
    return c * r

def rectangle_dimensions(min_vertex, max_vertex):
    """
    Input: min_vertex - a tuple representing the (latitude, longitude) of the bottom left vertex.
           max_vertex - a tuple representing the (latitude, longitude) of the top right vertex.
    Output: Returns the length and width of the rectangle in meters.
    """
    # Extract latitude and longitude for min and max vertices
    min_lat, min_lon = min_vertex
    max_lat, max_lon = max_vertex

    # Calculate length and width
    length = geopy.distance.geodesic((min_lat, min_lon), (max_lat, min_lon)).m
    width = geopy.distance.geodesic((min_lat, min_lon), (min_lat, max_lon)).m
    #length = haversine(min_lat, min_lon, max_lat, min_lon)  # Distance between bottom left and top left
    #width = haversine(min_lat, min_lon, min_lat, max_lon)  # Distance between bottom left and bottom right

    return length, width


In [58]:
%pip install segment-geospatial

In [59]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

In [60]:
m = leafmap.Map(center=[19.045589, 72.849362], zoom=19)
#m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m.add_basemap("SATELLITE")


In [61]:
#m

In [62]:
# bbox = m.user_roi_bounds()
# print(bbox)

In [63]:
# if m.user_roi_bounds() is not None:
#     bbox = m.user_roi_bounds()
# else:
bbox = [72.8488, 19.0384, 72.8604, 19.0484]

In [64]:
longitude = 0
latitude = 0

pointA = (bbox[0], bbox[1])
pointC = (bbox[2] , bbox[3])
pointB = (bbox[2] , bbox[3])
touplA = (longitude, latitude)
touplA  = rectangle_dimensions(pointA, pointB)




print(touplA)

(1294.5127019869128, 329.282713184461)


In [65]:
def create_geodataframe_from_bbox(min_point, max_point):
    """
    Create a GeoDataFrame from a bounding box defined by minimum and maximum points.

    Parameters:
    - min_point (tuple): The minimum point (longitude, latitude) of the bounding box.
    - max_point (tuple): The maximum point (longitude, latitude) of the bounding box.

    Returns:
    - GeoDataFrame: A GeoDataFrame with a single row containing the Polygon.
    """
    # Create the Polygon using the bounding box
    min_lon, min_lat = min_point
    max_lon, max_lat = max_point
    rectangle = Polygon([(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)])

    # Convert the Polygon into a GeoDataFrame
    gdf = gpd.GeoDataFrame([1], geometry=[rectangle], columns=['dummy'])
    return gdf

In [66]:
!pip install geopandas
import geopandas as gpd
gdf = create_geodataframe_from_bbox(pointA,pointB)

In [67]:
print(gdf)

   dummy                                           geometry
0      1  POLYGON ((72.84880 19.03840, 72.86040 19.03840...


In [68]:
import geopandas as gpd
teest_ =create_polygon_from_bbox(pointA, pointB)




bbox_polygon = Polygon([(bbox[0], bbox[1]), (bbox[0], bbox[3]),  (bbox[2], bbox[3]), (bbox[2], bbox[1])])



m.add_gdf(gdf=gdf, layer_name="Bounding Box")

# Display the map

# gdf = gpd.GeoDataFrame(index=[0], geometry=[bbox_polygon])

#m.add_gdf(gdf, layer_name="Bounding Box")
m

Map(center=[19.045589, 72.849362], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [69]:
def get_bbox_dimensions(min_lon, min_lat, max_lon, max_lat):
    """
    Calculate the dimensions of a bounding box.

    Parameters:
    - min_lon (float): Minimum longitude of the bounding box.
    - min_lat (float): Minimum latitude of the bounding box.
    - max_lon (float): Maximum longitude of the bounding box.
    - max_lat (float): Maximum latitude of the bounding box.

    Returns:
    - width (float): The width of the bounding box (longitude span).
    - height (float): The height of the bounding box (latitude span).
    """
    width = max_lon - min_lon
    height = max_lat - min_lat
    return width, height

# Example usage with your bounding box
def get_bbox_dimensions_in_meters(min_lon, min_lat, max_lon, max_lat):
    """
    Calculate the dimensions of a bounding box in meters.

    Parameters:
    - min_lon (float): Minimum longitude of the bounding box.
    - min_lat (float): Minimum latitude of the bounding box.
    - max_lon (float): Maximum longitude of the bounding box.
    - max_lat (float): Maximum latitude of the bounding box.

    Returns:
    - width (float): The width of the bounding box in meters (longitude span).
    - height (float): The height of the bounding box in meters (latitude span).
    """
    # Width: Distance between min and max longitude at the average latitude
    avg_lat = (min_lat + max_lat) / 2
    width = haversine(min_lon, avg_lat, max_lon, avg_lat)

    # Height: Distance between min and max latitude at the average longitude
    avg_lon = (min_lon + max_lon) / 2
    height = haversine(avg_lon, min_lat, avg_lon, max_lat)

    return width, height


In [70]:
print(bbox)

[72.8488, 19.0384, 72.8604, 19.0484]


In [71]:

width, height = get_bbox_dimensions_in_meters(pointA[1], pointA[0], pointB[1], pointB[0])

print(f"Width (Longitude Span): {width} meter")
print(f"Height (Latitude Span): {height} meter")

Width (Longitude Span): 1111.949266445872 meter
Height (Latitude Span): 1219.2692348906826 meter


In [72]:
get_basemaps().keys()
tms_to_geotiff(output="/root/satellite.tif", bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 0001/1120
Downloaded image 0002/1120
Downloaded image 0003/1120
Downloaded image 0004/1120
Downloaded image 0005/1120
Downloaded image 0006/1120
Downloaded image 0007/1120
Downloaded image 0008/1120
Downloaded image 0009/1120
Downloaded image 0010/1120
Downloaded image 0011/1120
Downloaded image 0012/1120
Downloaded image 0013/1120
Downloaded image 0014/1120
Downloaded image 0015/1120
Downloaded image 0016/1120
Downloaded image 0017/1120
Downloaded image 0018/1120
Downloaded image 0019/1120
Downloaded image 0020/1120
Downloaded image 0021/1120
Downloaded image 0022/1120
Downloaded image 0023/1120
Downloaded image 0024/1120
Downloaded image 0025/1120
Downloaded image 0026/1120
Downloaded image 0027/1120
Downloaded image 0028/1120
Downloaded image 0029/1120
Downloaded image 0030/1120
Downloaded image 0031/1120
Downloaded image 0032/1120
Downloaded image 0033/1120
Downloaded image 0034/1120
Downloaded image 0035/1120
Downloaded image 0036/1120
Downloaded image 0037/1120
D

In [73]:
def divide_picture_into_grids(width, height, grid_size=1000):
    """
    Divide a picture into grids of one kilometer (1 km) each.

    Parameters:
    - width (float): Width of the picture in meters.
    - height (float): Height of the picture in meters.
    - grid_size (float): Size of each grid cell in meters (default is 1000 meters).

    Returns:
    - list of tuples: Coordinates of the grid cells, each representing a one-kilometer square.
    """
    # Calculate the number of grid cells in each direction
    num_horizontal_grids = int(width / grid_size)
    num_vertical_grids = int(height / grid_size)

    # Initialize a list to store the coordinates of the grid cells
    grid_coordinates = []

    # Iterate over the picture and create grid cells
    for i in range(num_horizontal_grids):
        for j in range(num_vertical_grids):
            # Calculate the coordinates of the grid cell
            grid_x = i * grid_size
            grid_y = j * grid_size

            # Add the grid cell coordinates to the list
            grid_coordinates.append((grid_x, grid_y))

    return grid_coordinates

# Example usage
width_meters = 10000  # Assume the width of the picture is 10 km (10000 meters)
height_meters = 5000   # Assume the height of the picture is 5 km (5000 meters)
grid_size_meters = 1000  # Grid size is 1 km (1000 meters)

grid_coordinates = divide_picture_into_grids(width_meters, height_meters, grid_size_meters)
print("Grid coordinates (one kilometer squares):", grid_coordinates)

Grid coordinates (one kilometer squares): [(0, 0), (0, 1000), (0, 2000), (0, 3000), (0, 4000), (1000, 0), (1000, 1000), (1000, 2000), (1000, 3000), (1000, 4000), (2000, 0), (2000, 1000), (2000, 2000), (2000, 3000), (2000, 4000), (3000, 0), (3000, 1000), (3000, 2000), (3000, 3000), (3000, 4000), (4000, 0), (4000, 1000), (4000, 2000), (4000, 3000), (4000, 4000), (5000, 0), (5000, 1000), (5000, 2000), (5000, 3000), (5000, 4000), (6000, 0), (6000, 1000), (6000, 2000), (6000, 3000), (6000, 4000), (7000, 0), (7000, 1000), (7000, 2000), (7000, 3000), (7000, 4000), (8000, 0), (8000, 1000), (8000, 2000), (8000, 3000), (8000, 4000), (9000, 0), (9000, 1000), (9000, 2000), (9000, 3000), (9000, 4000)]


In [78]:
from PIL import Image
import numpy as np

In [79]:
def split_image_into_grid(image_path, grid_size):
    # Open the TIF image
    img = Image.open(image_path)

    # Convert the image to numpy array
    img_array = np.array(img)

    # Get the dimensions of the image
    height, width = img_array.shape[:2]

    # Calculate the size of each grid cell
    cell_height = height // grid_size
    cell_width = width // grid_size

    # List to store grid images
    grid_images = []

    # Split the image into grid cells
    for i in range(grid_size):
        for j in range(grid_size):
            # Calculate the coordinates of the grid cell
            top = i * cell_height
            bottom = (i + 1) * cell_height
            left = j * cell_width
            right = (j + 1) * cell_width

            # Extract the grid cell from the image
            grid_cell = img.crop((left, top, right, bottom))

            # Append the grid cell to the list
            grid_images.append(grid_cell)

    return grid_images
splittedimages = split_image_into_grid("/root/satellite.tif", 30)



In [80]:
print(len(splittedimages))

900


In [81]:
def save_images(image_path, grid_images, output_directory):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Save the original image
    original_image_name = os.path.basename(image_path)
    img_save_path = os.path.join(output_directory, original_image_name)
    img = Image.open(image_path)
    img.save(img_save_path)

    # Save the grid images
    for i, grid_image in enumerate(grid_images):
        grid_image_name = f"{original_image_name.split('.')[0]}_grid_{i}.png"
        grid_image_path = os.path.join(output_directory, grid_image_name)
        grid_image.save(grid_image_path)

    print("Images saved successfully!")


# Split the image into grid cells
# grid_images = split_image_into_grid("/root/satellite.tif", 100)

# Save both the original image and the grid images
save_images("/root/satellite.tif", splittedimages , "/root/splitted 30")

Images saved successfully!
